In [1]:
import sys
import climin
from functools import partial
import warnings
import os
sys.path.append('..')

import numpy as np
from scipy.stats import multinomial
from scipy.linalg.blas import dtrmm

import GPy
from GPy.util import choleskies
from GPy.core.parameterization.param import Param
from GPy.kern import Coregionalize
from GPy.likelihoods import Likelihood
from GPy.util import linalg

from likelihoods.bernoulli import Bernoulli
from likelihoods.gaussian import Gaussian
from likelihoods.categorical import Categorical
from likelihoods.hetgaussian import HetGaussian
from likelihoods.beta import Beta
from likelihoods.gamma import Gamma
from likelihoods.exponential import Exponential

from hetmogp.util import draw_mini_slices
from hetmogp.het_likelihood import HetLikelihood
from hetmogp.svmogp import SVMOGP
from hetmogp import util
from hetmogp.util import vem_algorithm as VEM

import matplotlib.pyplot as plt
from matplotlib.pyplot import gca
from matplotlib import rc, font_manager
from matplotlib import rcParams

warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

import pickle
import random
import time

import error_func
import click
import logging

In [2]:
Q = 1  # number of latent functions
mode = 'multi_output'
y_dim = 1
M = 100

x_high = np.load('../data/fertility/x_step1_mean.np')
x_low = np.load('../data/fertility/x_step2_mean.np')

y_high = np.load('../data/fertility/y_step1_mean.np')
y_low = np.load('../data/fertility/y_step2_mean.np')


x_high_new = np.zeros((x_high.shape[0], x_high.shape[1]+1))
x_high_new[:,0] = x_high[:,1].copy()
x_high_new[:,1] = x_high[:,0].copy()
x_high_new[:,2] = x_high[:,3].copy() - 15
x_high_new[:,3] = x_high[:,2].copy() - 15

x_low_new = np.zeros((x_low.shape[0], x_low.shape[1]+1))
x_low_new[:,0] = x_low[:,1].copy()
x_low_new[:,1] = x_low[:,0].copy()
x_low_new[:,2] = x_low[:,3].copy() - 15
x_low_new[:,3] = x_low[:,2].copy() - 15

x1_high, x2_high, x3_high, x4_high = x_high_new.copy(), x_high_new.copy(), x_high_new.copy(), x_high_new.copy()
Y1_high, Y2_high, Y3_high, Y4_high = y_high[:,0].reshape(-1, 1), y_high[:,1].reshape(-1, 1), y_high[:,2].reshape(-1, 1), y_high[:,3].reshape(-1, 1)

x1_low, x2_low, x3_low, x4_low = x_low_new.copy(), x_low_new.copy(), x_low_new.copy(), x_low_new.copy()
Y1_low, Y2_low, Y3_low, Y4_low = y_low[:,0].reshape(-1, 1), y_low[:,1].reshape(-1, 1), y_low[:,2].reshape(-1, 1), y_low[:,3].reshape(-1, 1)


# rnd_idx
random.seed(8)
size_random_idx = 1000
rnd_idx1 = random.sample(range(x1_high.shape[0]), size_random_idx)
rnd_idx2 = random.sample(range(x2_high.shape[0]), size_random_idx)

# sorted_rnd_idx
sorted_rnd_idx1 = np.sort(rnd_idx1)
sorted_rnd_idx2 = np.sort(rnd_idx2)

X_test1 = x1_high[sorted_rnd_idx1]
X_test2 = x2_high[sorted_rnd_idx2]

if y_dim == 1:
    Y_test = [Y1_high[sorted_rnd_idx1]]
elif y_dim == 2:
    Y_test = [Y1_high[sorted_rnd_idx1], Y2_high[sorted_rnd_idx2]]

x1_high_new = np.delete(x1_high, sorted_rnd_idx1, axis=0)
Y1_high_new = np.delete(Y1_high, sorted_rnd_idx1, axis=0)

x2_high_new = np.delete(x2_high, sorted_rnd_idx2, axis=0)
Y2_high_new = np.delete(Y2_high, sorted_rnd_idx2, axis=0)

# Normalising outputs
Y1_low_mean = Y1_low.mean().copy()
Y1_low_std = Y1_low.std().copy()

Y2_low_mean = Y2_low.mean().copy()
Y2_low_std = Y2_low.std().copy()

Y1_high_mean = Y1_high.mean().copy()
Y1_high_std = Y1_high.std().copy()

Y2_high_mean = Y2_high.mean().copy()
Y2_high_std = Y2_high.std().copy()

Y1_low_norm = (Y1_low - Y1_low_mean) / Y1_low_std
Y2_low_norm = (Y2_low - Y2_low_mean) / Y2_low_std

Y1_high_norm = (Y1_high_new - Y1_high_mean) / Y1_high_std
Y2_high_norm = (Y2_high_new - Y2_high_mean) / Y2_high_std

random.seed(8+4)
high_res_data_random_size = 80
rnd_idx_high_res = random.sample(range(x1_high_new.shape[0]), high_res_data_random_size)

# sorted_rnd_idx
sorted_rnd_idx_high_res = np.sort(rnd_idx_high_res)

if mode == 'multi_output':
    print('Multi output mode, y_dim:', y_dim)
    if y_dim == 1:
        X = [x1_high_new[sorted_rnd_idx_high_res], x1_low]
        Y = [Y1_high_norm[sorted_rnd_idx_high_res], Y1_low_norm]

        #  Creating inducing inputs using cluster centers of the original images
        X_ = np.vstack((X[0], X[1]))
        
        # Heterogeneous Likelihood Definition
        likelihoods_list = [Gaussian(sigma=1.), Gaussian(sigma=1.)]
    elif y_dim == 2:
        X = [x1_high_new[sorted_rnd_idx_high_res], x1_low,
             x2_high_new[sorted_rnd_idx_high_res], x2_low]
        Y = [Y1_high_norm[sorted_rnd_idx_high_res], Y1_low_norm,
             Y2_high_norm[sorted_rnd_idx_high_res], Y2_low_norm]
       
        #  Creating inducing inputs using cluster centers of the original images
        X_ = np.vstack((X[0], X[1], X[2], X[3]))
    
        # Heterogeneous Likelihood Definition
        likelihoods_list = [Gaussian(sigma=1.), Gaussian(sigma=1.), Gaussian(sigma=1.), Gaussian(sigma=1.)]

    ls_q = np.array(([2., 2.] * Q))
    var_q = np.array(([1.0]*Q))
else:
    print('Single output mode')
    if y_dim == 1:
        X = [x1_high_new[sorted_rnd_idx_high_res]]
        Y = [Y1_high_norm[sorted_rnd_idx_high_res]]
    elif y_dim == 2:
        X = [x2_high_new[sorted_rnd_idx_high_res]]
        Y = [Y2_high_norm[sorted_rnd_idx_high_res]]
        
    #  Creating inducing inputs using cluster centers of the original images
    X_ = np.vstack((X[0], X[0]))
    
    # Heterogeneous Likelihood Definition
    likelihoods_list = [Gaussian(sigma=1.)]
    ls_q = np.array(([2., 2.]*Q))
    var_q = np.array(([1.0]*Q))

if M > X_.shape[0]:
    logging.warning("More inducing points than X - setting Z to %d", X_.shape[0])
    M = X_.shape[0]
    
kmeans_X = KMeans(n_clusters=M, random_state=0).fit(X_)
kmeans_X.cluster_centers_.shape
Z = kmeans_X.cluster_centers_
Z[:,-1] = 1

likelihood = HetLikelihood(likelihoods_list)
Y_metadata = likelihood.generate_metadata()

D = likelihood.num_output_functions(Y_metadata)

W_list, _ = util.random_W_kappas(Q, D, rank=1, experiment=True)

# KERNELS
input_dim = 5

kern_list = util.latent_functions_prior(Q, lenghtscale=ls_q, variance=var_q, input_dim=input_dim)

model = SVMOGP(X=X, Y=Y, Z=Z, kern_list=kern_list, likelihood=likelihood, Y_metadata=Y_metadata, batch_size=50)

print(model)

print('model.kern_q0.lengthscale', model.kern_q0.lengthscale)

# Z should be fixed, is not implemented
model.Z.fix()

def transform_y(data_test):
    mpred, vpred = model.predict(data_test)
    mpred_transformed = []
    vpred_transformed = []
    if mode == 'multi_output':
        for i in range(len(mpred)):
            if i % 2 == 1:
                continue
            if i == 0:
                m_pred_star = mpred[i] * Y1_high_std + Y1_high_mean
                v_pred_star = vpred[i] * Y1_high_std * Y1_high_std
            elif i == 2:
                m_pred_star = mpred[i] * Y2_high_std + Y2_high_mean
                v_pred_star = vpred[i] * Y2_high_std * Y2_high_std
            else:
                raise Exception("Not implemented!")
            mpred_transformed.append(m_pred_star)
            vpred_transformed.append(v_pred_star)
    else:
        if y_dim == 1:
            m_pred_star = mpred[0] * Y1_high_std + Y1_high_mean
            v_pred_star = vpred[0] * Y1_high_std * Y1_high_std
        elif y_dim == 2:
            m_pred_star = mpred[0] * Y2_high_std + Y2_high_mean
            v_pred_star = vpred[0] * Y2_high_std * Y2_high_std

        mpred_transformed.append(m_pred_star)
        vpred_transformed.append(v_pred_star)

    return mpred_transformed, vpred_transformed

def transform_y_test_set(data_set):
    mpred_transformed = []
    vpred_transformed = []
    if model == 'multi_output':
        iters_count = y_dim
    else:
        iters_count = 1

    for i in range(iters_count):
        mpred, vpred = model.predict(data_set)
        if i == 0:
            if mode == 'multi_output':
                m_pred_star = mpred[i*2] * Y1_high_std + Y1_high_mean
                v_pred_star = vpred[i*2] * Y1_high_std * Y1_high_std
            else:
                if y_dim == 1:
                    m_pred_star = mpred[0] * Y1_high_std + Y1_high_mean
                    v_pred_star = vpred[0] * Y1_high_std * Y1_high_std
                elif y_dim == 2:
                    m_pred_star = mpred[0] * Y2_high_std + Y2_high_mean
                    v_pred_star = vpred[0] * Y2_high_std * Y2_high_std
        elif i == 1:
            assert mode == 'multi_output'
            m_pred_star = mpred[i*2] * Y2_high_std + Y2_high_mean
            v_pred_star = vpred[i*2] * Y2_high_std * Y2_high_std
        else:
            raise Exception("Not implemented!")

        mpred_transformed.append(m_pred_star)
        vpred_transformed.append(v_pred_star)

    return mpred_transformed, vpred_transformed


def error_calc(data_test, true_labels, calculate_snlp=False, test_mode=False, Y_train_snlp=None, 
               single_dim_only=False):
    if test_mode == False:
        m_pred_star, v_pred_star = transform_y(data_test)
    else:
# if test_mode is set, means that we want inference for test set data, x is shared, y is a list of y1, y2, ...
        m_pred_star, v_pred_star = transform_y_test_set(data_test)

    smse_error = error_func.smse(mu_star_list=m_pred_star, Y_test_list=true_labels)
    if calculate_snlp:
        if single_dim_only:
            snlp_error = error_func.snlp(mu_star_list=[m_pred_star[0]], var_star_list=[v_pred_star[0]], 
                                         Y_test_list=true_labels, Y_train_list=Y_train_snlp)
        else:
            snlp_error = error_func.snlp(mu_star_list=m_pred_star, var_star_list=v_pred_star, 
                                         Y_test_list=true_labels, Y_train_list=Y_train_snlp)

    else:
        snlp_error = None
        msll_error = None
    return smse_error, snlp_error

max_iter = 1000
print("Optimiser max iter: ", max_iter)


Multi output mode, y_dim: 1

Name : SVMOGP
Objective : 1651.0470505077521
Number of Parameters : 5657
Number of Optimization Parameters : 5657
Updates : True
Parameters:
  SVMOGP.              |      value  |  constraints  |  priors
  inducing_inputs      |   (100, 5)  |               |        
  m_u                  |   (100, 1)  |               |        
  L_u                  |  (5050, 1)  |               |        
  kern_q0.variance     |        1.0  |      +ve      |        
  kern_q0.lengthscale  |       (2,)  |      +ve      |        
  B_q0.W               |     (2, 1)  |               |        
  B_q0.kappa           |       (2,)  |      +ve      |        
model.kern_q0.lengthscale   index  |  SVMOGP.kern_q0.lengthscale  |  constraints  |  priors
  [0]    |                  2.00000000  |      +ve      |        
  [1]    |                  2.00000000  |      +ve      |        
Optimiser max iter:  1000


In [3]:
def callback(i):
    if i['n_iter'] % 100 == 0:
        print('svi - iteration ' + str(i['n_iter']) + '/' + str(max_iter) + ":" + (str(model.log_likelihood())))

        # Note that X_test is numpy array, and Y_test is a list
        if y_dim == 1:
            test_smse_error, test_snlp_error = error_calc(X_test1, Y_test, calculate_snlp=True, 
                                                                test_mode=True, 
                                                                Y_train_snlp=Y)

        elif y_dim == 2:                
            if mode == 'multi_output':
                test_smse_error_y1, test_snlp_error_y1 = error_calc(X_test1, [Y_test[0]], 
                                                                                       calculate_snlp=True, 
                                                                                       test_mode=True,
                                                                                       Y_train_snlp=[Y[0]],
                                                                                       single_dim_only=True)
                test_smse_error_y2, test_snlp_error_y2 = error_calc(X_test2, [Y_test[1]], 
                                                                                       calculate_snlp=True, 
                                                                                       test_mode=True,
                                                                                       Y_train_snlp=[Y[2]],
                                                                                       single_dim_only=True)
                test_smse_error = (test_smse_error_y1 + test_smse_error_y2) / 2.0
                test_snlp_error = (test_snlp_error_y1 + test_snlp_error_y2) / 2.0


            else:
                if y_dim == 1:
                    test_smse_error, test_snlp_error = error_calc(X_test1, Y_test, 
                                                                        calculate_snlp=True, 
                                                                        test_mode=True, Y_train_snlp=Y)
                elif y_dim == 2:
                    test_smse_error, test_snlp_error = error_calc(X_test2, [Y_test[1]], 
                                                                        calculate_snlp=True, 
                                                                        test_mode=True, Y_train_snlp=Y)

    if i['n_iter'] > max_iter:
        return True

    return False

start_time = time.time()
opt = climin.Adam(model.optimizer_array, model.stochastic_grad, step_rate=0.01, decay_mom1=1 - 0.9, 
                  decay_mom2=1 - 0.999)

opt.minimize_until(callback)
print("--- %s seconds ---" % (time.time() - start_time))
test_smse_error, test_snlp_error = error_calc(X_test1, Y_test, calculate_snlp=True, 
                                                                test_mode=True, 
                                                                Y_train_snlp=Y)

print("Test SMSE:", test_smse_error)
print("Test SNLP:", test_snlp_error)
print("***********************************************************************")

print(model)


svi - iteration 100/1000:[[-1457.20677999]]
svi - iteration 200/1000:[[-1218.87779029]]
svi - iteration 300/1000:[[-983.52022031]]
svi - iteration 400/1000:[[-871.14472836]]
svi - iteration 500/1000:[[-808.53632294]]
svi - iteration 600/1000:[[-803.52985688]]
svi - iteration 700/1000:[[-853.80240453]]
svi - iteration 800/1000:[[-831.94530008]]
svi - iteration 900/1000:[[-810.09994718]]
svi - iteration 1000/1000:[[-775.16090062]]
--- 2523.0613961219788 seconds ---
Test SMSE: [0.04392279]
Test SNLP: [-3.68148376]
***********************************************************************

Name : SVMOGP
Objective : 772.1801584682671
Number of Parameters : 5657
Number of Optimization Parameters : 5157
Updates : True
Parameters:
  SVMOGP.              |               value  |  constraints  |  priors
  inducing_inputs      |            (100, 5)  |     fixed     |        
  m_u                  |            (100, 1)  |               |        
  L_u                  |           (5050, 1)  |       